## Introduktion til `spaCy` 

"spaCy" indeholder forskellige sprogmodeller - herunder en dansk sprogmodel.

Overordnet virker spaCy ved, at man specificerer en sprogmodel samt nogen "processors", som modellen skal indeholde. 

SpaCy's sprogmodeller indeholder blandt andet:
- Tokenizer (inddeling i enkeltord)
- Lemmatizer (konvertering til navneform)
- Part-Of-Speech tagging (POS-tagging) (identificering af ordtyper)
- Dependency parsing (sætningskonstruktion)
- Named-Entity-Recognition (NER) (udledning af "named entities", fx personer og organisationer)

## Brug af spaCy i Python

1. Indlæs sprogmodel
2. Analysér tekstykke
3. Inspicér resultater

In [1]:
import spacy

#!python -m spacy download 'da_core_news_sm' # evt. installer sprogmodel

Når sprogmodellen er hentet, kan vi bruge den ved at indlæse modellen. Som standard indlæses modellen med alle processerne, men det er muligt at aktivere/deaktivere specifikke processer.

Efter modellen er defineret, kan man lade sprogmodellen analysere tekst.

In [2]:
nlp = spacy.load("da_core_news_sm") # Definerer model

doc = nlp('Politiet har givet borgerne råd') # Analyserer tekst med model

Når modellen anvendes på et stykke tekst, behandler den tekststykket med de forskellige processors, som er en del af sprogmodellen (som standard for dansk: tokenizer, part-of-speech tagging, lemmatizer og dependency parsing).

Outputtet (`doc`) indeholder de forskellige værdier, som er udledt af teksten, som attributes (et attribute for token, et for lemma, et for POS-tag osv.).

Vi kan fx visualisere sætningskonstruktionen med funktionen `displacy`:

In [3]:
spacy.displacy.render(doc, style='dep')

## Lemmatizing

Et ords "lemma" er dets grammatiske stamme (fx "er"->"være", "spiste"->"spise"). SpaCy's sprogmodeller indeholder typisk en indbygget ordbog til at finde stammen for de enkelte ord. Et ords "lemma" er gem under attributtet `.lemma_` for hvert ord:

In [10]:
for word in doc:
    print(f'{word.text:<15} {word.lemma_}')

Politiet        politi
har             have
givet           give
borgerne        borger
råd             råd


## Part-of-speech tags

SpaCy tagger automatisk hvert ord med sin ordklasse ("part-of-speech"-tag/POS-tag). Disse er gemt under attributtet `.pos_` for hvert ord:

In [11]:
for word in doc:
    print(f'{word.text:<15} {word.pos_}')

Politiet        NOUN
har             AUX
givet           VERB
borgerne        NOUN
råd             NOUN


Part-of-speech tagging virker ved, at modellen i forvejen er trænet på danske tekster, og derfor har "set" de forskellige ord i kontekst før. Som det kan ses, er modellen dog ikke perfekt (fx "trygge" er angivet som navneord (NOUN), selvom der her er tale om et tillægsord (ADJ)).

Part-of-speech tagging tillader fx at isolere visse ord i et stykke tekst:

In [12]:
keep_tags = ['NOUN', 'ADJ', 'PROPN']
keep_words = []

for word in doc:
    if word.pos_ in keep_tags:
        keep_words.append(word)

for word in keep_words:
    print(f'{word.text:<15} {word.pos_}')

Politiet        NOUN
borgerne        NOUN
råd             NOUN


## Dependency parsing

SpaCy laver også analyse af sætningskonstruktion (dependency parsing). Hvilken del af sætningen, som ordene er analyseret frem til, kan tilgås af attribut `.dep_`.

In [13]:
for word in doc:
    print(f'{word.text:<15} {word.dep_}')

Politiet        nsubj
har             aux
givet           ROOT
borgerne        obj
råd             obj


## Named entities

"Named entities" kan groft sagt forstås som "meningsfulde enheder" i teksten. Det kan fx være personer, organisationer eller steder. Ligesom ved part-of-speech tagging, fungerer "named entity recognition" ved, at modellen enten har set disse enheder før eller er bekendt med, hvordan sådanne enheder fremgår i sætningen (hvor er de i sætningskonstruktionen, hvilke ordklasser er de associeret med).

Alle ord i en tekst er ikke en "named entity". Named entities kan tilgås gennem attributtet `ents` for det behandlede stykke tekst (`doc`). Fra dette kan ses, hvilke enheder er udledt, og hvordan de er kategoriseret:

In [17]:
doc = nlp("Søs Marie Serup, politisk kommentator og tidligere særlig rådgiver for Løkke, fortalte i fredags i DR's nyhedspodcast 'Genstart' om hans evne til altid at komme tilbage i politik.")

for ent in doc.ents:
    print(f'{ent.text:<15} {ent.label_}')

Marie Serup     PER
Løkke           LOC
DR's            ORG


Denne sprogmodel arbejder med fire named entity tags:
- LOC: Steder
- ORG: Organisationer
- PER: Personer
- MISC: Andet

Af ovenstående ses, at modellen identificerer "DR" som en organisation, hvilket er meget passende. Derudover genkender den "Marie Serup" som person, men har udeladt fornavnet "Søs". Dog er "Løkke" fejlklassificeret som et sted (LOC).